In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson, map_eq_earth
from GMST import GMST_timeseries, GMST_regression, atm_heat_content, GMST_GISTEMP
from paths import path_results, path_samoc, path_data
from plotting import shifted_color_map, discrete_cmap
from constants import abs_zero, cp_air
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import xr_AREA

In [ ]:
from OHC import replace_OHC_year
from filters import bandpass
from xr_regression import xr_lintrend, xr_quadtrend

In [ ]:
gmst_ctrl = xr.open_dataarray(f'{path_samoc}/GMST/GMST_yrly_ctrl.nc')
gmst_lpd  = xr.open_dataarray(f'{path_samoc}/GMST/GMST_yrly_lpd.nc' )

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_ctrl.nc',
                         decode_times=False)
ctrl   = replace_OHC_year(ctrl  , 205)
lpd    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_lpd.nc',
                         decode_times=False)

In [ ]:
ctrl.OHC_global.plot()

In [ ]:
def variances(ts):
    for i, band in enumerate([(20,5), (40,10), (80,20)]):
        print(band, bandpass(ts-xr_quadtrend(ts), np.array(band)).std().values)
    print('')
    return
variances(ctrl.OHC_global)
variances(lpd .OHC_global)

variances(gmst_ctrl)
variances(gmst_lpd)


In [ ]:
def std_bandpass(tseries):
    if type(tseries)==xr.core.dataarray.DataArray:
        tseries = tseries-xr_quadtrend(tseries)
        tseries = tseries.values
    a = np.std(bandpass(tseries, np.array([20,5])))
    b = np.std(bandpass(tseries, np.array([100,20])))
    return a, b



f, ax = plt.subplots(2,2,figsize=(14,7),sharex='col', sharey=True)
for i in range(2):
    for j in range(2):
        ax[i,j].tick_params(labelsize=20)

        ax[i,j].spines['right'].set_visible(False)
        ax[i,j].spines['top'].set_visible(False)
    
    ax[i,0].set_yticks([2.75, 2, 1])
    ax[i,0].set_yticklabels(['obs', 'high\nres.', 'low\nres.'], fontsize=20)

ax[1,0].set_xlabel('GMST variance [K]', fontsize=30)

had1, had2 = std_bandpass(had_cmip5)
ctrl1, ctrl2 = std_bandpass(gmst_ctrl[-149:])
ctrl3, ctrl4 = std_bandpass(gmst_ctrl[:149])
lpd1, lpd2 = std_bandpass(gmst_lpd[-149:])
lpd3, lpd4 = std_bandpass(gmst_lpd[-2*149:-149])

ax[0,0].set_ylabel('decadal', fontsize=30)
ax[0,0].barh(2.75, had1 , 0.42, color='C1')
ax[0,0].barh(2.25, ctrl1, 0.42, color='C0')
ax[0,0].barh(1.75, ctrl3, 0.42, color='C0')
ax[0,0].barh(1.25, lpd1 , 0.42, color='C2')
ax[0,0].barh(0.75, lpd3 , 0.42, color='C2')


ax[1,0].barh(2.75, had2 , 0.42, color='C1')
ax[1,0].barh(2.25, ctrl2, 0.42, color='C0')
ax[1,0].barh(1.75, ctrl4, 0.42, color='C0')
ax[1,0].barh(1.25, lpd2,  0.42, color='C2')
ax[1,0].barh(0.75, lpd4,  0.42, color='C2')
ax[1,0].set_ylabel('multidecadal', fontsize=30)

ax[1,1].set_xlabel('global OHC variance [ZJ]', fontsize=30)

ctrl1, ctrl2 = std_bandpass(ctrl.OHC_global[-149:])
ctrl3, ctrl4 = std_bandpass(ctrl.OHC_global[:149])
lpd1, lpd2 = std_bandpass(lpd.OHC_global[-149:])
lpd3, lpd4 = std_bandpass(lpd.OHC_global[-2*149:-149])

ax[0,1].barh(2.25, ctrl1/1e21, 0.42, color='C0')
ax[0,1].barh(1.75, ctrl3/1e21, 0.42, color='C0')
ax[0,1].barh(1.25, lpd1 /1e21, 0.42, color='C2')
ax[0,1].barh(0.75, lpd3 /1e21, 0.42, color='C2')

ax[1,1].barh(2.25, ctrl2/1e21, 0.42, color='C0')
ax[1,1].barh(1.75, ctrl4/1e21, 0.42, color='C0')
ax[1,1].barh(1.25, lpd2 /1e21, 0.42, color='C2')
ax[1,1].barh(0.75, lpd4 /1e21, 0.42, color='C2')

print('test')
plt.tight_layout()
plt.savefig(f'{path_results}/poster/std_GMST_OHC', dpi=600)